In [29]:
import math
import pyprob
from pyprob import util, Model
from pyprob.distributions import Normal, Uniform, Categorical

pyprob.set_verbosity(1)

## PyProb 

* **PyProb** is a PyTorch-based probabilistic programming language
* It supports **inference compilation** with minimal intervention
* It uses a protocol (PPX) that allows **distributed** training and inference

## Primitive Stochastic Functions

* Built on top of **PyTorch's** distributions library
* API is, however, slightly different than Pyro's

### Drawing sample from unit Normal 

In [77]:
mu = 0.
sigma = 1.

In [78]:
normal = Normal(mu, sigma) # create a normal distribution object

In [79]:
x = normal.sample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(-0.0271)
log prob tensor(-0.9193)


* Just like in the Pyro examples, these distributions and sampling functions are backed by the underlying PyTorch implementation.

### Sampling 

In [80]:
x = Normal(mu, sigma).sample()
print(x)

tensor(2.6212)


* No need to explicitly pass a name to the random variable.

## The Ice-Cream Model

Let’s suppose we have a bunch of data with daily mean temperatures and cloud cover. We want to reason about how temperature interacts with whether it was sunny or cloudy. A simple stochastic function that does that is given by:

In [81]:
def weather():
    cloudy = Uniform(0, 1).sample()
    cloudy = 'cloudy' if cloudy < 0.3 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = Normal(mean_temp, scale_temp).sample()
    return cloudy, temp
    
for _ in range(3):
    print(weather())

('cloudy', tensor(43.7089))
('sunny', tensor(78.6348))
('sunny', tensor(71.9160))


In [82]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    return Normal(expected_sales, 10.0).sample()

In [83]:
ice_cream_sales()

tensor(36.1148)

# Inference: One Example

In [84]:
class GaussianUnknownMean(Model):
    def __init__(self):
        super().__init__(name="Gaussian with unknown mean") # give the model a name
        
        # prior mean and std
        self.prior_mean = 1
        self.prior_stdd = math.sqrt(5)
        self.likelhood_stdd = math.sqrt(2)

    # Needed to specifcy how the generative model is run forward
    def forward(self):
        # sample the (latent) mean variable to be inferred:
        mu = pyprob.sample(Normal(self.prior_mean, self.prior_stdd))

        # define the likelihood
        likelihood = Normal(mu, self.likelhood_stdd)

        # observation "placeholders"
        # these can have concrete values associated with them later when
        # conditioning on data.
        pyprob.observe(likelihood, name='observation-1')
        pyprob.observe(likelihood, name='observation-2')

        # return the latent quantity of interest
        return mu

# create an instance of the model
gum = GaussianUnknownMean()

* The probabilistic program above models **Gaussian with Unknown Mean**: our goal is to infer the expected value of `mu` above.
* PyProb supports multiple inference algorithms, including inference compilation (discussed later)

In [85]:
posterior = gum.posterior_distribution(
    num_traces=1000, # number of samples to be used
    inference_engine=pyprob.InferenceEngine.IMPORTANCE_SAMPLING, # the inference algorithm
    observe={'observation-1': 9, 'observation-2': 8} # conditioning on some observations
)

# inferring the mean and standard deviation of our `mu` latent variable
print("mean:", posterior.mean)
print("stddev:", posterior.stddev)

mean: tensor(6.7454)
stddev: tensor(0.7475)


* PyProb provides other inference algorithms too:
    * Importance sampling
    * Lightweight Metropolis-Hastings
    * Random Walk Metropolis-Hastings

# Inference Compilation

PyProb supports **inference compilation**. Let's apply that to the _Gaussian with Unknown Mean_ example we used in the last section. Recall that we defined a model for it:

In [86]:
gum

PyProb provides an inference engine to use in combination with a **learned inference network**.

First, we need to learn the inference network:

In [87]:
gum.learn_inference_network(
    num_traces=1000, # how many traces (executions of the generative model) to use to train the inference network
    observe_embeddings={
        'observation-1': {'dim': 10},
        'observation-2': {'dim': 10}
    }
)

Creating new inference network...
Initializing inference network...
Observable observation-1: observe embedding not specified, using the default FEEDFORWARD.
Observable observation-1: embedding depth not specified, using the default 2.
Observable observation-2: observe embedding not specified, using the default FEEDFORWARD.
Observable observation-2: embedding depth not specified, using the default 2.
New proposal layer for address: 16__forward__mu__Normal__1
Total number of parameters: 5,834
Train. time | Trace     | Init. loss| Min. loss | Curr. loss| T.since min | Traces/sec
0d:00:00:02 | 1,024     | +2.21e+00 | +2.16e+00 | +2.19e+00 | 0d:00:00:00 | 405.0                              


Once the inference network is **trained** with the traces produced by the generative model, we can use it to perform **inference**:

In [88]:
posterior = gum.posterior_distribution(
    num_traces=1000,
    inference_engine=pyprob.InferenceEngine.IMPORTANCE_SAMPLING_WITH_INFERENCE_NETWORK,
    observe={'observation-1': 9, 'observation-2': 8.4}
)

# inferring the mean and standard deviation of our `mu` latent variable
print("mean:", posterior.mean)
print("stddev:", posterior.stddev)

mean: tensor(7.4163)
stddev: tensor(1.0121)


# Inference Compilation: Solving Captchas

In [20]:
import numpy as np
import torch
from PIL import Image, ImageDraw, ImageFont

class MiniCaptcha(Model):
    _alphabet = [
        'A', 'b', 'C', 'd', 'E', 'f', 'G', 'h', 'I', 'j', 'K', 'l', 'M',
        'n',  'O', 'p', 'Q', 'r', 'S', 't', 'U', 'v', 'X', 'w', 'Y', 'z'
    ]
    
    _fonts = [
        'Ubuntu-B.ttf',
        'FFF_Tusj.ttf',
        'LobsterTwo-Regular.otf',
        'SHPinscher-Regular.otf'
    ]

    def __init__(self, alphabet=_alphabet, fonts=_fonts, noise=0.1):
        self._alphabet = alphabet
        self._letter_probs = [1/len(alphabet) for i in range(len(alphabet))]
        self._font_probs = [1/len(fonts) for i in range(len(fonts))]
        self._noise = noise
        super().__init__('MiniCaptcha')
        
    def render_canvas(self, text, font, size=18, height=28, width=28, x=6, y=6):
        pil_font = ImageFont.truetype(font, size=size)
        text_width, text_height = pil_font.getsize(text)
        canvas = Image.new('RGB', [height, width], (255, 255, 255))
        draw = ImageDraw.Draw(canvas)
        draw.text((x, y), text, font=pil_font, fill='#000000')
        return canvas

    def render(self, text, font, size=18, height=28, width=28, x=6, y=6):
        canvas = self.render_canvas(text, font, size, height, width, x, y)
        return torch.from_numpy(1 - (np.asarray(canvas) / 255.0))[:, :, 0].unsqueeze(0).float()
    
    def forward_canvas(self):
        letter_id = Categorical(self._letter_probs).sample()
        font_id = Categorical(self._font_probs).sample()
        
        image = self.render_canvas(self._alphabet[letter_id], self._fonts[font_id])
        return image

    def forward(self):
        letter_id = pyprob.sample(Categorical(self._letter_probs))
        font_id = pyprob.sample(Categorical(self._font_probs))
        
        image = self.render(self._alphabet[letter_id], self._fonts[font_id]).view(-1)
        likelihood = Normal(image, self._noise)
        pyprob.observe(likelihood, name='query_image')
        return letter_id
            
captcha_model = MiniCaptcha()

In [3]:
import IPython
import io

for _ in range(10):
    canvas = captcha_model.forward_canvas()
    bytes = imgByteArr = io.BytesIO()
    canvas.save(bytes, format='PNG')

    IPython.display.display(IPython.display.Image(bytes.getvalue()))

In [21]:
def one_hot(dim, i):
    t = torch.zeros(dim)
    t.narrow(0, i, 1).fill_(1)
    return t

epsilon = 1e-8

true_posteriors = [
    Categorical(one_hot(len(MiniCaptcha._alphabet), i) + epsilon) for i in range(len(MiniCaptcha._alphabet))
]

print(true_posteriors)

[Categorical(num_categories: 26, probs:tensor([1.0000e+00, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        1.0000e-08, 1.0000e-08])), Categorical(num_categories: 26, probs:tensor([1.0000e-08, 1.0000e+00, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        1.0000e-08, 1.0000e-08])), Categorical(num_categories: 26, probs:tensor([1.0000e-08, 1.0000e-08, 1.0000e+00, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08, 1.0000e-08,
        1.0000e

In [22]:
# Training the inference network

captcha_model.learn_inference_network(
    num_traces=1000,
    observe_embeddings={'query_image': {'dim': 32,
                                        'reshape': [1, 28, 28],
                                        'embedding': pyprob.ObserveEmbedding.CNN2D5C}}
)

Creating new inference network...
Initializing inference network...
New proposal layer for address: 12__forward__letter_id__Categorical(len_probs:26)_...
New proposal layer for address: 28__forward__font_id__Categorical(len_probs:4)__1
Total number of parameters: 455,138
Train. time | Trace     | Init. loss| Min. loss | Curr. loss| T.since min | Traces/sec
0d:00:00:09 | 1,024     | +4.64e+00 | +4.62e+00 | +4.63e+00 | 0d:00:00:01 | 109.0                              


In [31]:
import random

alphabet = MiniCaptcha._alphabet
choose_font = lambda: MiniCaptcha._fonts[random.randint(0, len(MiniCaptcha._fonts) - 1)]

test_images = [captcha_model.render(letter, choose_font()).view(-1) for letter in alphabet]

posteriors = []
estimates = []

for i in range(len(alphabet)):
    posterior = captcha_model.posterior_distribution(
        1000,
        pyprob.InferenceEngine.IMPORTANCE_SAMPLING_WITH_INFERENCE_NETWORK,
        observe={'query_image': test_images[i]}
    )
    
    posteriors.append(posterior)
    estimates.append(alphabet[int(posterior.mode)])

correct_estimates = sum([1 if estimates[i] == alphabet[i] else 0 for i in range(len(alphabet))])
accuracy = correct_estimates / len(alphabet)

kl_divergence = float(sum([pyprob.distributions.Distribution.kl_divergence(
    util.empirical_to_categorical(posterior, max_val=len(alphabet) - 1), true_posterior)
                           for (posterior, true_posterior) in zip(posteriors, true_posteriors)]))



print("Accuracy:", accuracy)
print("KL-divergence of all posterior distributions:", kl_divergence)

Accuracy: 1.0
KL-divergence of all posterior distributions: 9.536744300930877e-07
